<a href="https://colab.research.google.com/github/EllouziMedAmin/DSWithPytorch/blob/main/vectorDB_Quran.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install chromadb

In [2]:
import re
from langchain.text_splitter import TextSplitter, RecursiveCharacterTextSplitter, SentenceTransformersTokenTextSplitter
import chromadb
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction
from uuid import uuid4

In [3]:
def max_token_length(txt_list:list):
    max_length = 0
    for txt in txt_list:
        token_count = len(re.findall(r'\w+', txt))
        if token_count > max_length:
            max_length = token_count
    return f"Max Token Length: {max_length} tokens"

In [4]:
lorem_ipsum = """
Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua.
Augue mauris augue neque gravida in fermentum et. Felis bibendum ut tristique et egestas quis ipsum suspendisse ultrices.
Duis tristique sollicitudin nibh sit amet commodo nulla facilisi nullam. Pretium aenean pharetra magna ac placerat.
Quis risus sed vulputate odio ut. Consectetur adipiscing elit duis tristique sollicitudin nibh. Nec nam aliquam sem et.
Sed blandit libero volutpat sed cras. Faucibus pulvinar elementum integer enim neque volutpat ac.
Mi in nulla posuere sollicitudin aliquam ultrices sagittis. Eget egestas purus viverra accumsan. Diam vel quam elementum pulvinar etiam non quam.
Arcu cursus euismod quis viverra nibh cras. A scelerisque purus semper eget duis at. Lectus vestibulum mattis ullamcorper velit sed ullamcorper.
Eget felis eget nunc lobortis mattis aliquam faucibus purus in. Elit scelerisque mauris pellentesque pulvinar pellentesque habitant.
Ornare suspendisse sed nisi lacus sed. Interdum velit laoreet id donec ultrices. Ipsum a arcu cursus vitae congue mauris rhoncus aenean vel.
Faucibus nisl tincidunt eget nullam non nisi. Urna condimentum mattis pellentesque id nibh. Tellus in hac habitasse platea dictumst vestibulum.
Eget est lorem ipsum dolor. Enim eu turpis egestas pretium aenean pharetra magna ac placerat. Ac turpis egestas integer eget aliquet nibh.
Vivamus arcu felis bibendum ut tristique et egestas. Nisi lacus sed viverra tellus in hac habitasse platea dictumst. Odio ut enim blandit volutpat maecenas volutpat.
Turpis egestas sed tempus urna et pharetra pharetra massa. Dui nunc mattis enim ut tellus elementum sagittis vitae et. Nunc sed velit dignissim sodales ut eu.
Aliquam ut porttitor leo a diam sollicitudin tempor id. At quis risus sed vulputate odio ut enim blandit volutpat. Gravida quis blandit turpis cursus in hac habitasse platea dictumst.
Sit amet nulla facilisi morbi tempus iaculis urna. Diam maecenas sed enim ut sem viverra aliquet eget. Turpis egestas pretium aenean pharetra.
At varius vel pharetra vel turpis nunc eget lorem. Integer quis auctor elit sed. Eget nunc lobortis mattis aliquam. Et magnis dis parturient montes nascetur ridiculus mus mauris vitae.
Sollicitudin nibh sit amet commodo. Integer quis auctor elit sed vulputate mi sit amet mauris. Est placerat in egestas erat imperdiet.
Ornare quam viverra orci sagittis eu volutpat odio facilisis mauris. Semper quis lectus nulla at volutpat diam. Amet volutpat consequat mauris nunc congue nisi.
Ipsum nunc aliquet bibendum enim facilisis gravida neque convallis a. Et pharetra pharetra massa massa ultricies. Nunc eget lorem dolor sed viverra ipsum nunc aliquet bibendum.
"""


In [5]:
# chroma default sentence model "all-MiniLM-L6-v2"
# https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2
# max input length: 256 characters
model_max_chunk_length = 256
token_splitter = SentenceTransformersTokenTextSplitter(
    tokens_per_chunk=model_max_chunk_length,
    model_name="all-MiniLM-L6-v2",
    chunk_overlap=0
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
lorem_ipsum_tokens = token_splitter.split_text(lorem_ipsum)

In [7]:
max_token_length(lorem_ipsum_tokens)

'Max Token Length: 110 tokens'

In [8]:
text_path = "/content/quran.txt"
with open(text_path, "r", encoding="utf-8") as f:
    text_raw = f.read()

In [11]:
text_raw[:1000]

'\ufeff001|001|In the name of Allah, the Gracious, the Merciful.\n001|002|Praise be to Allah, Lord of the Universes.\n001|003|The Most Gracious, the Most Merciful.\n001|004|Sovereign of the Day of Judgment.\n001|005|It is You we worship, and You we ask for help.\n001|006|Guide us along the Straight Path.\n001|007|The path of those You have blessed, not of the condemned, nor of the astrayed.\n002|001|Alif. Lam. Meem.\n002|002|This is the Book, beyond any doubt, a guide for the righteous.\n002|003|Those who believe in the unseen, perform the prayer, and spend from what We provide for them.\n002|004|And those who believe in what was revealed to you, and in what was revealed before you, and are certain of the Hereafter.\n002|005|These are guided by their Lord, and these are the successful.\n002|006|As for those who disbelieve, whether you warn them or not, it’s the same for them; they will not believe.\n002|007|Allah sealed their hearts and hearing, and over their vision is a veil. They wi

In [12]:
character_splitter = RecursiveCharacterTextSplitter(
    separators=['\n    \n', '\n\n', '\n', '. '],
    chunk_size=1000,
    chunk_overlap=0,
)
text_splitted = character_splitter.split_text(text_raw)
print(f"Total number of splitted chunks: {len(text_splitted)}")

Total number of splitted chunks: 877


In [13]:
max_token_length(text_splitted)


text_tokens = []
for text in text_splitted:
    text_tokens.extend(token_splitter.split_text(text))
print(f"Total number of tokens: {len(text_tokens)}")

Total number of tokens: 1147


In [14]:
max_token_length(text_tokens)

'Max Token Length: 193 tokens'

In [15]:
embedding_fn = SentenceTransformerEmbeddingFunction(
    model_name="all-MiniLM-L6-v2",
    device="cuda"
)


In [16]:
len(embedding_fn(text_tokens[0])[0])

384

In [17]:
chroma_db=chromadb.Client()
chroma_collection = chroma_db.create_collection(
    name="quran_collection",
    embedding_function=embedding_fn
)

In [18]:
ids=[str(uuid4()) for _ in range(len(text_tokens))]
ids[:5]

['a1d55d32-f729-48cc-bb29-2347fcc2150a',
 '878bbb6b-5f82-44a7-b0b3-e36b799e0fde',
 '4ec34e96-6f81-4dbb-b8a4-fccd2808b00b',
 'de76f343-9cb5-4f26-81c4-07576d0b3b40',
 '9abac9e6-a192-4bfe-a900-d8a7f6a4c9d1']

In [19]:
chroma_collection.add(documents=text_tokens, ids=ids)

In [20]:
res=chroma_collection.query(query_texts=["What are some important teachings of Prophet Muhammad (PBUH)?"], n_results=10)

In [21]:
res['documents']

[['048 | 029 | muhammad is the messenger of allah. those with him are stern with the unbelievers, compassionate among themselves. you see them bowing and prostrating, seeking grace and approval from allah. their marks are on their faces from the effects of prostration. this is their description in the torah and the gospel : like a seed that sprouts, grows, strengthens, thickens, and then stands firm on its stem, delighting the sowers. he enrages the unbelievers with them. allah has promised those among them who believe and do righteous deeds, forgiveness and a great reward. 049 | 001 | o you who believe, don ’ t place your opinions above those of allah and his messenger. allah is all - hearing, all - knowing. 049 | 002 | o you who have faith, don ’ t raise your voices above the prophet ’ s, and don ’ t speak to him in a loud voice as you speak loudly to one another, lest your deeds become nullified without your awareness.',
  'prophet.',
  'many a prophet did we send among the ancients